In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

model = ChatOpenAI()

In [4]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    evaluation_score: float

In [5]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [6]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [13]:
def evaluate_blog(state: BlogState) -> BlogState:

    content = state['content']

    prompt = f'Evaluate the following blog content that how well Outline: {state["outline"]} is covered in \n Content: {content} \n evaluation score out of 10'

    evaluation = model.invoke(prompt).content

    state['evaluation_score'] = evaluation

    return state

In [14]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()


In [15]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': "I. Introduction \n    A. Brief overview of AI and its increasing role in various industries worldwide \n    B. Introduction to AI in India and its rapid growth in recent years \n\nII. Historical background of AI in India \n    A. Early developments in AI research and adoption in India \n    B. Key milestones and advancements in AI technology in India \n\nIII. Current landscape of AI in India \n    A. Overview of AI startups and companies in India \n    B. Adoption of AI in various sectors such as healthcare, finance, e-commerce, and agriculture \n    C. Government initiatives and policies to promote AI in India \n\nIV. Factors driving the growth of AI in India \n    A. Availability of talent and skilled workforce in the field of AI \n    B. Increasing investments in AI research and development by both domestic and international companies \n    C. Rising demand for AI solutions to improve efficiency and productivity \n\nV. Challenges and obst

In [10]:
print(final_state['outline'])

I. Introduction
    A. Definition of AI
    B. Overview of AI technology
    C. Importance of AI in India

II. History of AI in India
    A. Initial adoption of AI technology in India
    B. Growth of AI startups in India
    C. Government initiatives to promote AI in India

III. Current state of AI in India
    A. Major industries adopting AI in India
    B. Role of AI in improving efficiency and productivity in Indian businesses
    C. Challenges faced by AI adoption in India

IV. Future prospects of AI in India
    A. Expected growth of AI industry in India
    B. Potential impact of AI on job market in India
    C. Role of AI in shaping India's digital economy

V. Ethical and social implications of AI in India
    A. Concerns about AI replacing human jobs
    B. Ethics of AI in decision-making processes
    C. Ensuring responsible AI implementation in India

VI. Conclusion
    A. Recap of importance of AI in India
    B. Summary of opportunities and challenges in AI adoption in Ind

In [11]:
print(final_state['content'])

The Rise of AI in India

I. Introduction
A. Definition of AI
Artificial Intelligence, or AI, refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding.
B. Overview of AI technology
AI technology encompasses a wide range of applications, including machine learning, natural language processing, robotics, and more. AI systems are designed to mimic human cognitive functions and have the ability to learn from data.
C. Importance of AI in India
AI technology has the potential to transform various industries and sectors in India, improving efficiency, productivity, and innovation. The adoption of AI in India can help drive economic growth and competitive advantage in the global market.

II. History of AI in India
A. Initial adoption of AI technology in India
India has been quick to embrace AI technology, with companies and startups across various sector

In [16]:
print(final_state['evaluation_score'])

I would rate this blog content a 8 out of 10. It provides a comprehensive overview of AI in India, covering historical background, current landscape, factors driving growth, challenges faced, future outlook, and government initiatives. The content is well-structured with clear headings and subheadings. However, it could benefit from more specific examples and case studies to support the points made. Additionally, a deeper analysis of the potential implications of AI on the job market and economy in India could further enhance the content.
